In [1]:
%%capture
!pip install bertopic

In [9]:
with open("C:/Users/portable_laura/Documents/coursM1/Mémoire/Sources/french_stopwords.txt",'r',encoding='utf-8') as s:
    stop = s.read().split()

In [ ]:
import pandas as pd
from bertopic import BERTopic
import json
mot_cible = "inond"
df_mot_cible = pd.read_csv(f"C:/Users/portable_laura/Desktop/memoire_prod_finale/data/{mot_cible}.csv")

In [ ]:
#targets_canal = df_canal["date"]
#targets_inon = df_inon["date"]
targets = df_mot_cible["date"]

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop]
    return " ".join(filtered_words)

#===================================
df_mot_cible["texte_total_cleaned"] = df_mot_cible["texte_total"].apply(remove_stopwords)
docs_mot_cible = df_mot_cible["texte_total_cleaned"]

## Première modélisation

In [ ]:
topic_model_mot_cible = BERTopic(verbose=True)
topics_mot_cible, _ = topic_model_mot_cible.fit_transform(docs_mot_cible)

2025-09-01 00:45:21,781 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/29 [00:00<?, ?it/s]

2025-09-01 00:46:25,497 - BERTopic - Embedding - Completed ✓
2025-09-01 00:46:25,498 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-01 00:46:30,673 - BERTopic - Dimensionality - Completed ✓
2025-09-01 00:46:30,675 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-01 00:46:30,777 - BERTopic - Cluster - Completed ✓
2025-09-01 00:46:30,788 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-09-01 00:46:31,514 - BERTopic - Representation - Completed ✓


In [ ]:
pd.reset_option('display.max_colwidth')

topic_mode_mot_cible.get_topic_info().head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,0,916,0_travaux_barrage_fr_demande,"[travaux, barrage, fr, demande, 000, publics, ...",[manche ose engager terrain techni demande grâ...
1,1,12,1_nantes_loire_lechalas_amont,"[nantes, loire, lechalas, amont, port, kilomtr...",[erreur arrêté exact conduit penser levées pro...


Tentative de nouvelle visualisation, il est ici possible de choisir les topics souhaités

In [ ]:
topics_per_class_mot_cible = topic_model_mot_cible.topics_per_class(docs_mot_cible, classes=targets)
topics_per_class_mot_cible['Class'] = pd.to_datetime(topics_per_class_mot_cible['Class'])
topics_per_class_mot_cible['Class'] = topics_per_class_mot_cible['Class'].dt.year
fig_unsupervised_mot_cible = topic_model_mot_cible.visualize_topics_per_class(topics_per_class_mot_cible, top_n_topics=10)
fig_unsupervised_mot_cible

411it [00:35, 11.49it/s]


Tri des ordonnées par ordre croissant pour un résultat plus cohérent

In [ ]:
df_mot_cible["année"] = pd.to_datetime(df_mot_cible["date"]).dt.year
topic_model_mot_cible = BERTopic(verbose=True)
topics_mot_cible, _ = topic_model_mot_cible.fit_transform(docs_mot_cible, y=df_mot_cible["année"])

2025-09-01 00:47:07,716 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/29 [00:00<?, ?it/s]

2025-09-01 00:48:09,132 - BERTopic - Embedding - Completed ✓
2025-09-01 00:48:09,134 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-01 00:48:13,903 - BERTopic - Dimensionality - Completed ✓
2025-09-01 00:48:13,906 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-01 00:48:14,004 - BERTopic - Cluster - Completed ✓
2025-09-01 00:48:14,009 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-09-01 00:48:14,932 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model_mot_cible.get_topic_info().head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,360,-1_travaux_barrage_fr_000,"[travaux, barrage, fr, 000, etat, publics, dem...",[crédits levées rive prévoir rive gauche levée...
1,0,72,0_travaux_publics_barrage_navigation,"[travaux, publics, barrage, navigation, demand...",[escompter moyennequ novacancespar an impossib...
2,1,59,1_marins_marine_btiments_anglais,"[marins, marine, btiments, anglais, marin, min...",[quecouenespérions beauhoupderatés dernièremen...
3,2,51,2_gauche_extrme_politique_applaudissements,"[gauche, extrme, politique, applaudissements, ...",[proclame droit dis interdit internationale sé...
4,3,46,3_fr_chap_000_crdit,"[fr, chap, 000, crdit, travaux, chapitre, budg...",[mérite agricole 3 127 fr pourvu crédits moyen...
5,4,31,4_guerre_remise_prsidence_demandant,"[guerre, remise, prsidence, demandant, arme, o...",[engageretnous engager voiesanssavoirexactemen...
6,5,30,5_avril_1930_droit_1920,"[avril, 1930, droit, 1920, article, demande, n...",[biens intérêts privés saurait considéré établ...
7,6,27,6_grisou_explosion_ingnieur_accident,"[grisou, explosion, ingnieur, accident, feu, g...",[points présent servi thème compagnie défendre...
8,7,24,7_crolard_autorisation_eau_permissionnaire,"[crolard, autorisation, eau, permissionnaire, ...",[margaine plaisir crolard plaisir margaine pro...
9,8,19,8_margaine_rhin_alsace_kembs,"[margaine, rhin, alsace, kembs, canal, grand, ...",[enquête bref délai construction dubarragede k...


In [ ]:
topic_model_mot_cible.visualize_barchart(top_n_topics=20)


In [ ]:
fig = topic_model_mot_cible.visualize_barchart(top_n_topics=20)
fig.write_image(f"C:/Users/portable_laura/Desktop/memoire_prod_finale/data/barchart_topics_{mot_cible}.png")

In [ ]:


topics_per_class_mot_cible['Class'] = pd.to_datetime(topics_per_class_mot_cible['Class'])
topics_per_class_mot_cible['Class'] = topics_per_class_mot_cible['Class'].dt.year
topics_sorted_mot_cible = topics_per_class_mot_cible.sort_values(by='Class')

fig_unsupervised = topic_model_mot_cible.visualize_topics_per_class(topics_sorted_mot_cible, top_n_topics=10)
fig_unsupervised

IndexError: index 0 is out of bounds for axis 0 with size 0

#### Tentative de vectorisation TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from bertopic import BERTopic

chemin_stopwords = "C:/Users/portable_laura/Documents/coursM1/Mémoire/Sources/french_stopwords.txt"
with open(chemin_stopwords,'r',encoding="utf-8") as fichier_stop:
    custom_stopwords = fichier_stop.read().strip()
#ou alternative manuelle:
#custom_stopwords=["fr","demande","applaudissements","gauche"]


#On créé un vectoriseur TF-IDF avec des stopwords que l'on rajoute manuellement
vectorizer = TfidfVectorizer(stop_words=custom_stopwords)

X = vectorizer.fit_transform(docs_mot_cible)


topic_model_test2 = BERTopic(verbose=True)

docs_list = []
for doc_idx in range(X.shape[0]):
    doc = ' '.join(vectorizer.inverse_transform(X[doc_idx])[0])
    docs_list.append(doc)

topics_barrage, _ = topic_model_test2.fit_transform(X)

InvalidParameterError: The 'stop_words' parameter of TfidfVectorizer must be a str among {'english'}, an instance of 'list' or None. Got "a\nabord\nabsolument\nafin\nah\nai\naie\naient\naies\nailleurs\nainsi\nait\nallaient\nallo\nallons\nallô\nalors\nanterieur\nanterieure\nanterieures\napres\naprès\nas\nassez\nattendu\nau\naucun\naucune\naucuns\naujourd\naujourd'hui\naupres\nauquel\naura\naurai\nauraient\naurais\naurait\nauras\naurez\nauriez\naurions\naurons\nauront\naussi\nautant\nautre\nautrefois\nautrement\nautres\nautrui\naux\nauxquelles\nauxquels\navaient\navais\navait\navant\navec\navez\naviez\navions\navoir\navons\nayant\nayez\nayons\nb\nbah\nbas\nbasee\nbat\nbeau\nbeaucoup\nbien\nbigre\nbon\nboum\nbravo\nbrrr\nc\ncar\nce\nceci\ncela\ncelle\ncelle-ci\ncelle-là\ncelles\ncelles-ci\ncelles-là\ncelui\ncelui-ci\ncelui-là\ncelà\ncent\ncependant\ncertain\ncertaine\ncertaines\ncertains\ncertes\nces\ncet\ncette\nceux\nceux-ci\nceux-là\nchacun\nchacune\nchaque\ncher\nchers\nchez\nchiche\n

In [ ]:
topic_model_mot_cible.visualize_topics()

